# Raw Data Visualization 

In [10]:
library(tidyverse)
library(here)
library(jsonlite)
library(ggpubr)
library(ggridges)
library(patchwork)
library(janitor)

here::i_am('README.md')

here() starts at /home/datascience/cleaned_aplc_soils_project



In [11]:
dl_dat <- read_csv(here('data/processed/dl_survey_remote_sense_data.csv'),show_col_types = FALSE)
apl_dat <- read_csv(here('data/processed/apl_survey_remote_sense_data.csv'),show_col_types = FALSE)

In [12]:
dl_dat |>
    select(polygon_id,LOCPRESENT,CAT,SHP,GHP,SAD,GAD) |>
    mutate(across(c(LOCPRESENT,CAT,SHP,GHP,SAD,GAD), ~as.factor(.))) |> summary()

   polygon_id                LOCPRESENT                           CAT       
 Min.   :     194   {'present': 1}:80347   {'Adult': 1}             :45430  
 1st Qu.: 6370812   {'present': 2}:27959   {'Swarm': 1}             :16224  
 Median :18340158   {'present': 3}: 8784   {'Hopper': 1}            :11150  
 Mean   :16149088   {'present': 4}: 4684   {'Hopper': 1, 'Adult': 1}:10673  
 3rd Qu.:23582905   {'present': 5}: 2300   {'Band': 1}              : 8325  
 Max.   :30876401   {'present': 6}: 1620   {'Adult': 2}             : 5969  
                    (Other)       : 4996   (Other)                  :32919  
               SHP                   GHP                      SAD       
 {'no': 1}       :70760   {'no': 1}    :75404   {'no': 1}       :37861  
 {'no': 2}       :17095   {'no': 2}    :24233   {'isolated': 1} :21520  
 {'group': 1}    : 7374   {'no': 3}    : 6705   {'scattered': 1}:11277  
 {'no': 3}       : 4971   {'no': 4}    : 3640   {'group': 1}    : 9759  
 {'group': 2}    : 

In [13]:
# cols

cols <- c('LOCPRESENT','CAT','SHP','GHP','SAD','GAD','REPRELIAB')

# Function to parse the JSON-like strings
parse_json_like <- function(x) {
  x <- gsub("'", "\"", x)  # Replace single quotes with double quotes
  fromJSON(x)
}

# Function to unnest a JSON-like column
unnest_json_column <- function(data, column_name) {
  data |>
    select(polygon_id,all_of(column_name)) |>
    mutate(temp_col = map(.data[[column_name]], parse_json_like)) |>
    unnest_longer(temp_col, indices_to = "id") |>
    rename(!!paste0(column_name, "_id") := id, !!paste0(column_name, "_value") := temp_col)
}

# Unnest each JSON-like column individually
unnested_data_list <- list()
for (col in cols) {
  unnested_data <- unnest_json_column(dl_dat, col)
  unnested_data_list[[col]] <- unnested_data
}

# Combine the unnested columns into a single data frame
result <- reduce(unnested_data_list, full_join, by = "polygon_id")  |>
    select(!any_of(cols))


In [14]:
dl_dat2 <- dl_dat |>
    select(!all_of(cols)) |>
    left_join(result,by='polygon_id')  |>
    select(!c('Unnamed: 0',geometry_x,geometry_y)) |>
    janitor::clean_names()

# Column descriptions
- polygon_id = the id associated with the polygon in the overall polygon geojson. This is an important grouping variable as all values are the counts within that polygon
  
- startdate = this is the min, median, and max time stamp for all dates within this specific polygon

- [x,y] = the x and y centroid of the specific polygon id

- total_* columns = this is the total number of counts per locust category

- columns ending in '_median' this is the IRISIC soil grid data

- columns ending in '_ev' this is the australian soil grid data

- columns starting in 'bio' this is the bioclim data

- cat_id = wether the specific row is about adults, hoppers, swarms, or bands (this is originall four datafames I concatted together)
    - cat_value = count within that specific cat (stands for category) 
      
- locpresent_* columns = reported presence/absence of locusts

- shp_* columns = solitarious hopper observations

- ghp_* columns = gregarious hoppper observations

- sad_* columns = solitarius adult observations

- gad_* columns = gregarious adult obsercations

- repreliab_* columns = is the number of reliable or not oberservations within polygon_id

other columns should be self explanatory 

In [15]:
env_var_cols = c('x0_bdod_median', 'x10_silt_median', 'x1_cec_median', 'x2_cfvo_median', 'x3_clay_median', 
                 'x4_nitrogen_median', 'x5_ocd_median', 'x6_ocs_single_band', 'x7_phh2o_median', 'x8_sand_median', 
                 'x9_soc_median','bio01', 'bio02', 'bio03', 'bio04', 'bio05', 'bio06', 'bio07', 'bio08', 'bio09', 'bio10', 
                 'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19', 'elevation', 'p_hc_p_hc_000_005_ev',
                 'p_hc_p_hc_005_015_ev', 'tree_canopy_cover')

hopper_dat <- dl_dat2 |>
    select(polygon_id,x,y,total_shp_count,total_ghp_count,shp_id,shp_value,ghp_id,ghp_value,all_of(env_var_cols))

Below is how I classify what an 'outbreak' is. It is roughly based on the `shp` and `ghp` columns.

Basically, if you are have a higher denisty rating than 'low' in `ghp_id` I considered it an outbreak, but there are some exceptions here.

In [16]:

hopper_classification_table <- hopper_dat |>
    select(shp_id,ghp_id) |>
    distinct() |>
    mutate(hopper_classification = case_when(shp_id == 'group' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'low' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'group' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'group' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'isolated' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'isolated' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'isolated' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'isolated' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'isolated_group' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'isolated_scattered' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'low' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'isolated_scattered_group' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'no' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'no' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'no' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'no' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'scattered' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'scattered' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'scattered' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'scattered' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'low' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'scattered_group' & ghp_id == 'unkown' ~ 'non-outbreak',
                                             shp_id == 'unkown' & ghp_id == 'high' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'low' ~ 'non-outbreak',
                                             shp_id == 'unkown' & ghp_id == 'low_high' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'low_medium' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'low_medium_high' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'medium' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'medium_high' ~ 'outbreak',
                                             shp_id == 'unkown' & ghp_id == 'no' ~ 'non-outbreak',
                                             shp_id == 'unkown' & ghp_id == 'unkown' ~ 'unkown', TRUE ~ NA))
                                             

In [17]:
env_dat <- hopper_dat |>
    select(all_of(env_var_cols),polygon_id,x,y)

head(env_dat)

x0_bdod_median,x10_silt_median,x1_cec_median,x2_cfvo_median,x3_clay_median,x4_nitrogen_median,x5_ocd_median,x6_ocs_single_band,x7_phh2o_median,x8_sand_median,⋯,bio17,bio18,bio19,elevation,p_hc_p_hc_000_005_ev,p_hc_p_hc_005_015_ev,tree_canopy_cover,polygon_id,x,y
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>
110.0,267.0,348.5,55.0,532.0,7864.916,502.0,75,61.0,199.0,⋯,155,424,162,0.0000,NA,NA,11.5,194,-77.83333,18.00000
130.5,438.5,259.5,121.5,455.5,3408.503,421.5,70,57.5,104.5,⋯,211,508,233,133.0000,NA,NA,21.5,5459,-77.66667,18.16667
128.5,460.0,247.0,109.5,414.0,3433.750,387.0,62,56.0,130.5,⋯,173,565,183,312.5294,NA,NA,22.5,8394,-77.58333,18.00000
117.5,361.0,400.5,66.5,444.0,5421.890,500.5,81,54.0,199.5,⋯,267,481,346,716.0000,NA,NA,41.5,16627,-77.33333,18.33333
147.0,322.5,273.5,69.5,412.5,3128.395,325.0,51,71.5,265.5,⋯,90,252,90,33.0000,NA,NA,8.5,27418,-77.00000,18.00000
130.5,336.5,298.0,140.5,372.5,4342.000,421.5,71,58.0,284.5,⋯,336,357,357,161.0000,NA,NA,43.0,27451,-77.00000,18.33333


In [18]:
hopper_env_dat <- hopper_dat |>
    select(!all_of(env_var_cols)) |>
    select(!c(x,y)) |>
    left_join(hopper_classification_table,by=c('shp_id','ghp_id')) |>
    select(!c(shp_id,ghp_id)) |>
    group_by(polygon_id,hopper_classification) |>
    summarize(n = n(),polygon_id=first(polygon_id))|>
    pivot_wider(names_from = hopper_classification,values_from = n,values_fill = 0) |>
    left_join(env_dat,by='polygon_id')
              

`summarise()` has grouped output by 'polygon_id'. You can override using the
`.groups` argument.


In [19]:
# Human readable names 

rs_metadata <- read_csv(here('data/metadata/remote_sensed_variable_descriptions.csv'),show_col_types = FALSE) |>
    select(variable_name,short_description,conversion_factor) |>
    mutate(short_description = tolower(short_description),
          conversion_factor = replace_na(conversion_factor,replace = 1))  |>
  mutate(short_description_file_name = str_replace_all(short_description, "[^[:alnum:] ]", ""),
         short_description_file_name = str_replace_all(short_description_file_name, " ", "_"))

In [20]:

env_var <- gsub("x[0-9]+_", "", env_var_cols)

create_plot <- function(data, x_var, y_var, conversion_rate, title, y_label) {
    ggplot(data, aes_string(x = x_var, y = paste0(".data[[i]] * ", conversion_rate))) +
        geom_point(pch=21, alpha=0.02) +
        geom_smooth(se=FALSE) +
        theme_pubr() +
        xlab('') +
        ylab(y_label) +
        ggtitle(title)
}

for (i in env_var) {
    print(paste0('Now computing ', i))
    
    env_variable_name <- rs_metadata |>
        filter(variable_name == i) |>
        pull(short_description)
    
    conversion_rate <- rs_metadata |>
        filter(variable_name == i) %>%
        pull(conversion_factor) %>%
        ifelse(is.na(.), 1, .)

    file_name <- rs_metadata |>
        filter(variable_name == i) %>%
        pull(short_description_file_name)

    
    outbreak_plot <- create_plot(hopper_env_dat, 'outbreak', i, conversion_rate, 'Outbreaks', env_variable_name)
    nonoutbreak_plot <- create_plot(hopper_env_dat, '`non-outbreak`', i, conversion_rate, 'Non-outbreaks', env_variable_name)
    unknown_plot <- create_plot(hopper_env_dat, 'unkown', i, conversion_rate, 'Unknown', env_variable_name)
    
    ridgeline_plot <- hopper_env_dat %>%
        ungroup() %>%
        pivot_longer(c(`non-outbreak`, outbreak, unkown), names_to = 'locust_category', values_to = 'number_observations') %>%
        ggplot(aes(y = locust_category, x = number_observations)) +
            geom_density_ridges() +
            scale_x_continuous(trans = 'sqrt') +
            ylab('') +
            xlab('number of observations (sqrt)') +
            ggtitle('observation distribution') +
            theme_pubr()
    
    combined_plots <- (outbreak_plot + nonoutbreak_plot) / (unknown_plot + ridgeline_plot) + plot_annotation(title = env_variable_name)
    
    suppressMessages(suppressWarnings(
        ggsave(combined_plots,
               file = here(paste0('output/spatial_modeling/raw_data_viz/point_data_viz/', file_name, '.png')),
               width = 10, height = 10)
    ))
}

[1] "Now computing bdod_median"


ERROR: Error in eval(expr, envir, enclos): object 'hopper_env_dat2' not found


# lets just try to build a predictive model

In [79]:
names(hopper_env_dat)
env_vars <- gsub("x[0-9]+_", "", (env_var))

# Create a list of elements to remove
remove_elements <- c("p_hc_p_hc_000_005_ev", "p_hc_p_hc_005_015_ev")

# Filter the list to remove specific elements
env_vars <- env_vars[!env_vars %in% remove_elements]
env_vars

[1] "polygon_id"           "non-outbreak"         "outbreak"            
 [4] "unkown"               "x0_bdod_median"       "x10_silt_median"     
 [7] "x1_cec_median"        "x2_cfvo_median"       "x3_clay_median"      
[10] "x4_nitrogen_median"   "x5_ocd_median"        "x6_ocs_single_band"  
[13] "x7_phh2o_median"      "x8_sand_median"       "x9_soc_median"       
[16] "bio01"                "bio02"                "bio03"               
[19] "bio04"                "bio05"                "bio06"               
[22] "bio07"                "bio08"                "bio09"               
[25] "bio10"                "bio11"                "bio12"               
[28] "bio13"                "bio14"                "bio15"               
[31] "bio16"                "bio17"                "bio18"               
[34] "bio19"                "elevation"            "p_hc_p_hc_000_005_ev"
[37] "p_hc_p_hc_005_015_ev" "tree_canopy_cover"    "x"                   
[40] "y"

[1] "bdod_median"       "silt_median"       "cec_median"       
 [4] "cfvo_median"       "clay_median"       "nitrogen_median"  
 [7] "ocd_median"        "ocs_single_band"   "phh2o_median"     
[10] "sand_median"       "soc_median"        "bio01"            
[13] "bio02"             "bio03"             "bio04"            
[16] "bio05"             "bio06"             "bio07"            
[19] "bio08"             "bio09"             "bio10"            
[22] "bio11"             "bio12"             "bio13"            
[25] "bio14"             "bio15"             "bio16"            
[28] "bio17"             "bio18"             "bio19"            
[31] "elevation"         "tree_canopy_cover"

In [91]:
mod_dat <- hopper_env_dat |>
    ungroup() |>
    select(any_of(env_var_cols),outbreak,x,y) |>
    select(!c(p_hc_p_hc_000_005_ev,p_hc_p_hc_005_015_ev)) |>
    sf::st_as_sf(coords = c("x", "y")) |>
    drop_na()

names(mod_dat)
head(mod_dat)

[1] "x0_bdod_median"     "x10_silt_median"    "x1_cec_median"     
 [4] "x2_cfvo_median"     "x3_clay_median"     "x4_nitrogen_median"
 [7] "x5_ocd_median"      "x6_ocs_single_band" "x7_phh2o_median"   
[10] "x8_sand_median"     "x9_soc_median"      "bio01"             
[13] "bio02"              "bio03"              "bio04"             
[16] "bio05"              "bio06"              "bio07"             
[19] "bio08"              "bio09"              "bio10"             
[22] "bio11"              "bio12"              "bio13"             
[25] "bio14"              "bio15"              "bio16"             
[28] "bio17"              "bio18"              "bio19"             
[31] "elevation"          "tree_canopy_cover"  "outbreak"          
[34] "geometry"

x0_bdod_median x10_silt_median x1_cec_median x2_cfvo_median x3_clay_median
1 110.0          267.0           348.5          55.0          532.0         
2 130.5          438.5           259.5         121.5          455.5         
3 128.5          460.0           247.0         109.5          414.0         
4 117.5          361.0           400.5          66.5          444.0         
5 147.0          322.5           273.5          69.5          412.5         
6 130.5          336.5           298.0         140.5          372.5         
  x4_nitrogen_median x5_ocd_median x6_ocs_single_band x7_phh2o_median
1 7864.916           502.0         75                 61.0           
2 3408.503           421.5         70                 57.5           
3 3433.750           387.0         62                 56.0           
4 5421.890           500.5         81                 54.0           
5 3128.395           325.0         51                 71.5           
6 4342.000           421.5         71                 58.0           
  x8_sand_median geometry                   ⋯ bio14 bio15 bio16 bio17 bio18
1 199.0          POINT (-77.83333 18)       ⋯ 48    44    528   155   424  
2 104.5          POINT (-77.66667 18.16667) ⋯ 64    45    729   211   508  
3 130.5          POINT (-77.58333 18)       ⋯ 49    52    737   173   565  
4 199.5          POINT (-77.33333 18.33333) ⋯ 77    40    718   267   481  
5 265.5          POINT (-77 18)             ⋯ 29    67    434    90   252  
6 284.5          POINT (-77 18.33333)       ⋯ 80    34    691   336   357  
  bio19 elevation tree_canopy_cover outbreak geometry                  
1 162     0.0000  11.5              0        POINT (-77.83333 18)      
2 233   133.0000  21.5              0        POINT (-77.66667 18.16667)
3 183   312.5294  22.5              0        POINT (-77.58333 18)      
4 346   716.0000  41.5              0        POINT (-77.33333 18.33333)
5  90    33.0000   8.5              0        POINT (-77 18)            
6 357   161.0000  43.0              0        POINT (-77 18.33333)

In [92]:
library(tidymodels)
library(doParallel)
library(ranger)

set.seed(123)
good_folds <- spatial_block_cv(mod_dat, v = 5)

Warning message in spatial_block_cv(mod_dat, v = 5):
“`spatial_block_cv()` expects your data to have an appropriate coordinate reference system (CRS).
ℹ If possible, try setting a CRS using `sf::st_set_crs()`.
ℹ Otherwise, `spatial_block_cv()` will assume your data is in projected coordinates.”


In [93]:

# Create a list of elements to remove
remove_elements <- c("p_hc_p_hc_000_005_ev", "p_hc_p_hc_005_015_ev")

# Filter the list to remove specific elements
env_var_cols2 <- env_var_cols[!env_var_cols %in% remove_elements]


env_vars_str <- paste(env_var_cols2, collapse = " + ")


# Create the full formula string
formula_str <- paste("outbreak ~", env_vars_str)

# Convert the string to a formula

model_formula <- as.formula(formula_str)
model_formula

outbreak ~ x0_bdod_median + x10_silt_median + x1_cec_median + 
    x2_cfvo_median + x3_clay_median + x4_nitrogen_median + x5_ocd_median + 
    x6_ocs_single_band + x7_phh2o_median + x8_sand_median + x9_soc_median + 
    bio01 + bio02 + bio03 + bio04 + bio05 + bio06 + bio07 + bio08 + 
    bio09 + bio10 + bio11 + bio12 + bio13 + bio14 + bio15 + bio16 + 
    bio17 + bio18 + bio19 + elevation + tree_canopy_cover

In [94]:

# Define the model specification with importance and set the engine
rf_spec <- rand_forest(
    mode = 'regression',
    engine = 'ranger',
    trees = 2000) %>% 
    set_engine('ranger', importance = "impurity")

# Define the workflow
form <- model_formula

rf_wf <- workflow() %>%
    add_formula(form) %>%
    add_model(rf_spec)

# Perform cross-validation
set.seed(123)  # For reproducibility
cv_results <- fit_resamples(
    rf_wf,
    resamples = good_folds,
    metrics = metric_set(rmse, rsq),
    control = control_resamples(save_pred = TRUE)
)

# Collect metrics
cv_metrics <- collect_metrics(cv_results)

# Final model fit on the full dataset
final_rf <- rf_wf %>%
    fit(data = mod_dat)


In [90]:
show_notes(.Last.tune.result)

unique notes:
────────────────────────────────────────────────────────────────────────────────
Error: Missing data in columns: x0_bdod_median, x10_silt_median, x1_cec_median, x2_cfvo_median, x3_clay_median, x4_nitrogen_median, x5_ocd_median, x6_ocs_single_band, x7_phh2o_median, x8_sand_median, x9_soc_median, bio01, bio02, bio03, bio04, bio05, bio06, bio07, bio08, bio09, bio10, bio11, bio12, bio13, bio14, bio15, bio16, bio17, bio18, bio19, elevation, tree_canopy_cover.


In [96]:
final_rf

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Formula
Model: rand_forest()

── Preprocessor ────────────────────────────────────────────────────────────────
outbreak ~ x0_bdod_median + x10_silt_median + x1_cec_median + 
    x2_cfvo_median + x3_clay_median + x4_nitrogen_median + x5_ocd_median + 
    x6_ocs_single_band + x7_phh2o_median + x8_sand_median + x9_soc_median + 
    bio01 + bio02 + bio03 + bio04 + bio05 + bio06 + bio07 + bio08 + 
    bio09 + bio10 + bio11 + bio12 + bio13 + bio14 + bio15 + bio16 + 
    bio17 + bio18 + bio19 + elevation + tree_canopy_cover

── Model ───────────────────────────────────────────────────────────────────────
Ranger result

Call:
 ranger::ranger(x = maybe_data_frame(x), y = y, num.trees = ~2000,      importance = ~"impurity", num.threads = 1, verbose = FALSE,      seed = sample.int(10^5, 1)) 

Type:                             Regression 
Number of trees:                  2000 
Sample size:              